In [3]:
!pip install pydantic==1.10.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 2.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.12
    Uninstalling pydantic-1.10.12:
      Successfully uninstalled pydantic-1.10.12


In [17]:
!pip install docarray

  Using cached docarray-0.40.0-py3-none-any.whl.metadata (36 kB)
Using cached docarray-0.40.0-py3-none-any.whl (270 kB)


In [6]:
import openai
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [45]:
%set_env OPENAI_API_KEY=

env: OPENAI_API_KEY=


In [9]:
prompt = ChatPromptTemplate.from_template(
    "tell me a joke about {topic}"
)
model = ChatOpenAI()
parser = StrOutputParser()

In [10]:
chain = prompt | model | parser

In [14]:
chain.invoke({'topic':'wife'})

'Why did the wife run away with the vacuum cleaner?\n\nBecause it sucked all the joy out of her marriage!'

In [15]:
########

In [18]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
chain=prompt | model | parser

In [24]:
chain.invoke({"context":lambda x: retriever.get_relevant_documents(x["question"]),"question":"what bear likes to eat"})

'There is no information provided in the context to answer the question about what bear likes to eat.'

In [25]:
#######above did not work 

In [26]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [27]:
from langchain.schema.runnable import RunnableMap

In [31]:
chain = RunnableMap({"context":lambda x: retriever.get_relevant_documents(x['question']),
                    "question":lambda x: x["question"]}) | prompt | model | parser

In [32]:
chain.invoke({"question":"where did harrision work"})

'Harrison worked at Kensho.'

In [33]:
######Bind

In [34]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [35]:
prompt = ChatPromptTemplate.from_messages([
    ("human","{input}")
])

In [36]:
model = ChatOpenAI().bind(functions=functions)

In [37]:
chain = prompt | model

In [38]:
chain.invoke({"input":"what is the weather in Hyderabad india"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"HYD"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 65, 'total_tokens': 81}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-a18f0eb9-b2ae-4ce7-a890-c9a2e584f463-0')

In [39]:
chain.invoke({"input":"what is the my name"})

AIMessage(content="I'm sorry, but I am not able to access or retrieve your name. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 63, 'total_tokens': 87}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-7ba56c07-8010-4cce-bc51-f1923b0922cd-0')

In [40]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [41]:
model = ChatOpenAI().bind(functions=functions)

In [42]:
chain = prompt | model

In [43]:
chain.invoke({"input":"how did IPL Team RCB did today"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"RCB"}', 'name': 'sports_search'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 99, 'total_tokens': 116}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-054af0d0-1796-4899-9ee7-1084d8f3ffcc-0')